# Packages

In [1]:
import cv2

from google.colab import files
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import clear_output, HTML
clear_output()

# Prep

In [2]:
path = '/usr/local/share/jupyter/nbextensions/google.colab'
!cp -r {path}/* .
!rm -r {path}
!ln -s /content {path}
# change base tag

def change_base_url():
  eval_js("""
  var base = document.createElement('base')
  base.href = 'https://localhost:8080/nbextensions/google.colab/'
  document.head.prepend(base)
  """)
# make it run automatically in every cell
get_ipython().events.register('pre_run_cell', change_base_url)

fpath_img1 = 'bg_removal.jpeg'
!gdown 1IA1eJzkyVZtrLQvqnx6G1uNNZM6OFOLx -O "$fpath_img1"

fpath_img2 = 'od_sample2.jpg'
!gdown 19UxDH_g3EHwEnz1WvLSlXe6D8ypgIwNC -O "$fpath_img2"

clear_output()

# Human Body Segmentation

In [3]:
#@title Template Image
html_code = HTML('''
<!-- 載入tfjs 套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@1.2"></script>
<!-- 載入BodyPix 人體分割模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/body-pix@2.0"></script>

<!-- 顯示原圖的canvas -->
<canvas id='canvas1'></canvas>
<!-- 呈現分析結果的canvas -->
<canvas id='canvas2'></canvas>

<script>
// canvas 物件宣告
const canvas1 = document.getElementById('canvas1');
const canvas2 = document.getElementById('canvas2');
// canvas 繪製物件宣告
var ctx = canvas1.getContext('2d')

// js 影像物件宣告
var myImage = new Image();
// 指定本地端範例影像路徑
myImage.src = "bg_removal.jpeg";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // canvas 繪製原圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const net = await bodyPix.load();

  // 模型推論分析
  const segmentation = await net.segmentPerson(canvas1);
  console.log(segmentation);
  
  // 繪製模型分析結果
  const maskBackground = true;
  const foregroundColor = {r: 0, g: 0, b: 0, a: 0}; // 前景mask顏色
  const backgroundColor = {r: 0, g: 0, b: 0, a: 255}; // 背景mask顏色
  const backgroundDarkeningMask = bodyPix.toMask(
      segmentation, foregroundColor, backgroundColor);
  
  const opacity = 0.7; // 底圖影像的不透明度
  const maskBlurAmount = 3; // 人體與背景區塊邊界模糊程度
  const flipHorizontal = false; //關閉水平翻轉前處理
  // 繪製結果
  bodyPix.drawMask(
      canvas2, canvas1, backgroundDarkeningMask, opacity, maskBlurAmount, flipHorizontal);
}
</script>
''')

html_code

In [4]:
#@title Custom Image
uploaded = files.upload()
fname = list(uploaded.keys())[0]

html_code2 = HTML('''
<!-- 載入tfjs 套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@1.2"></script>
<!-- 載入BodyPix 人體分割模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/body-pix@2.0"></script>

<!-- 顯示原圖的canvas -->
<canvas id='canvas1'></canvas>
<!-- 呈現分析結果的canvas -->
<canvas id='canvas2'></canvas>

<script>
// canvas 物件宣告
const canvas1 = document.getElementById('canvas1');
const canvas2 = document.getElementById('canvas2');
// canvas 繪製物件宣告
var ctx = canvas1.getContext('2d')

// js 影像物件宣告
var myImage = new Image();
// 指定本地端範例影像路徑
myImage.src = "%s";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // canvas 繪製原圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const net = await bodyPix.load();

  // 模型推論分析
  const segmentation = await net.segmentPerson(canvas1);
  console.log(segmentation);
  
  // 繪製模型分析結果
  const maskBackground = true;
  const foregroundColor = {r: 0, g: 0, b: 0, a: 0}; // 前景mask顏色
  const backgroundColor = {r: 0, g: 0, b: 0, a: 255}; // 背景mask顏色
  const backgroundDarkeningMask = bodyPix.toMask(
      segmentation, foregroundColor, backgroundColor);
  
  const opacity = 0.7; // 底圖影像的不透明度
  const maskBlurAmount = 3; // 人體與背景區塊邊界模糊程度
  const flipHorizontal = false; //關閉水平翻轉前處理
  // 繪製結果
  bodyPix.drawMask(
      canvas2, canvas1, backgroundDarkeningMask, opacity, maskBlurAmount, flipHorizontal);
}
</script>
''' % (fname))

html_code2

Saving 1.jpeg to 1.jpeg


In [11]:
#@title WebCam Image
html_code3 = HTML('''
<!-- 載入tfjs 套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@1.2"></script>
<!-- 載入BodyPix 人體分割模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/body-pix@2.0"></script>

<!-- 顯示原圖的canvas -->
<canvas id='canvas1' height="0" width="0"></canvas>
<!-- 呈現分析結果的canvas -->
<canvas id='canvas2' height="0" width="0"></canvas>

<script>
// canvas 物件宣告
const canvas1 = document.getElementById('canvas1');
const canvas2 = document.getElementById('canvas2');
// canvas 繪製物件宣告
var ctx = canvas1.getContext('2d')

// webcam 攝影
async function takePhoto() {
  // 讀取模型
  const net = await bodyPix.load();

  const div = document.createElement('div'); // 空行
  const capture = document.createElement('button'); // 攝影按鈕
  capture.textContent = 'Capture'; // 攝影按鈕文字
  div.appendChild(capture);

  const video = document.createElement('video'); // 宣告影片物件
  video.style.display = 'block';
  const stream = await navigator.mediaDevices.getUserMedia({video: true}); // 宣告串流物件

  document.body.appendChild(div);
  div.appendChild(video);
  video.srcObject = stream; // 使影片物件顯示串流內容
  await video.play();

  // 調整Video Element 大小
  google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

  // 等待點擊攝影按鈕
  await new Promise((resolve) => capture.onclick = resolve);

  // canvas 繪製底圖
  canvas1.width = video.videoWidth;
  canvas1.height = video.videoHeight;
  ctx.drawImage(video, 0, 0);

  // 停止 webcamera
  stream.getVideoTracks()[0].stop();
  div.remove();

  // 模型推論分析
  const segmentation = await net.segmentPerson(canvas1);
  console.log(segmentation);

  // 繪製模型分析結果
  const maskBackground = true;
  const foregroundColor = {r: 0, g: 0, b: 0, a: 0}; // 前景mask顏色
  const backgroundColor = {r: 0, g: 0, b: 0, a: 255}; // 背景mask顏色
  const backgroundDarkeningMask = bodyPix.toMask(
      segmentation, foregroundColor, backgroundColor);
  
  const opacity = 0.7; // 底圖影像的不透明度
  const maskBlurAmount = 3; // 人體與背景區塊邊界模糊程度
  const flipHorizontal = false; //關閉水平翻轉前處理
  // 繪製結果
  bodyPix.drawMask(
      canvas2, canvas1, backgroundDarkeningMask, opacity, maskBlurAmount, flipHorizontal);
}

takePhoto()
</script>
''')

html_code3

In [6]:
#@title WebCam Video
html_code4 = HTML('''
<head>
  <!-- 載入tfjs 套件 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@1.2"></script>
  <!-- 載入BodyPix 人體分割模型 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/body-pix@2.0"></script>
</head>

<center>
  <!-- 串流video -->
  <video autoplay playsinline muted id="webcam"></video>
  <!-- 顯示原圖的canvas -->
  <canvas hidden id='canvas1' height="0" width="0"></canvas>
  <!-- 呈現分析結果的canvas -->
  <canvas id='canvas2' height="0" width="0"></canvas>
</center>

<script>
async function app() {
  // 讀取模型
  const net = await bodyPix.load();

  const webcamElement = document.getElementById('webcam'); // 宣告影片物件

  // canvas 物件宣告
  const canvas1 = document.getElementById('canvas1');
  const canvas2 = document.getElementById('canvas2');
  // canvas 繪製物件宣告
  var ctx = canvas1.getContext('2d')

  let showResult = async function () {
    // canvas 繪製底圖
    canvas1.width = webcamElement.videoWidth;
    canvas1.height = webcamElement.videoHeight;
    ctx.drawImage(webcamElement, 0, 0);

    // 模型推論分析
    const segmentation = await net.segmentPerson(canvas1);
    // console.log(segmentation);

    // 繪製模型分析結果
    const maskBackground = true;
    const foregroundColor = {r: 0, g: 0, b: 0, a: 0}; // 前景mask顏色
    const backgroundColor = {r: 0, g: 0, b: 0, a: 255}; // 背景mask顏色
    const backgroundDarkeningMask = bodyPix.toMask(
        segmentation, foregroundColor, backgroundColor);
    
    const opacity = 0.7; // 底圖影像的不透明度
    const maskBlurAmount = 3; // 人體與背景區塊邊界模糊程度
    const flipHorizontal = false; //關閉水平翻轉前處理
    // 繪製結果
    bodyPix.drawMask(
        canvas2, canvas1, backgroundDarkeningMask, opacity, maskBlurAmount, flipHorizontal);

    // 設定每偵顯示秒數
    setTimeout(function () {
        showResult();
    }, 300);
  }
  let setupWebcam = function () {
      return new Promise((resolve, reject) => {
          const navigatorAny = navigator;
          navigator.getUserMedia = navigator.getUserMedia ||
              navigatorAny.webkitGetUserMedia || navigatorAny.mozGetUserMedia ||
              navigatorAny.msGetUserMedia;
          if (navigator.getUserMedia) {
              navigator.getUserMedia({
                      video: true
                  },
                  (stream) => {
                      webcamElement.srcObject = stream;
                      webcamElement.addEventListener('loadeddata', () => resolve(),
                          false);
                  },
                  (err) => reject(err));
          } else {
              reject("getUserMedia failed");
          }
      });
  }
  setupWebcam().then(
      () => {
          showResult();
      },
      (err) => {
          console.log(err);
      }
  )
}
app();
</script>
''')

html_code4

# Human Body Part Segmentation

In [7]:
#@title Template Image
html_code5 = HTML('''
<!-- 載入tfjs 套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@1.2"></script>
<!-- 載入BodyPix 人體分割模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/body-pix@2.0"></script>

<!-- 顯示原圖的canvas -->
<canvas id='canvas1'></canvas>
<!-- 呈現分析結果的canvas -->
<canvas id='canvas2'></canvas>
<canvas id='canvas3'></canvas>

<script>
// canvas 物件宣告
const canvas1 = document.getElementById('canvas1');
const canvas2 = document.getElementById('canvas2');
const canvas3 = document.getElementById('canvas3');
// canvas 繪製物件宣告
var ctx = canvas1.getContext('2d')

// js 影像物件宣告
var myImage = new Image();
// 指定本地端範例影像路徑
myImage.src = "od_sample2.jpg";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // canvas 繪製原圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const net = await bodyPix.load();

  // 模型推論分析
  const partSegmentation = await net.segmentMultiPersonParts(canvas1);
  console.log(partSegmentation);
  
  // 繪製模型分析結果
  const coloredPartImage = bodyPix.toColoredPartMask(partSegmentation);
  const opacity = 0.7; // 底圖影像的不透明度
  const flipHorizontal = false; //關閉水平翻轉前處理
  const maskBlurAmount = 0; // 人體與背景區塊邊界模糊程度
  const pixelCellWidth = 10.0; // 部位拼豆框pixel-wise寬度

  // 繪製mask結果
  bodyPix.drawMask(
    canvas2, canvas1, coloredPartImage, opacity, maskBlurAmount, flipHorizontal);

  // 繪製拼豆結果
  bodyPix.drawPixelatedMask(
    canvas3, canvas1, coloredPartImage, opacity, maskBlurAmount, flipHorizontal, pixelCellWidth);
}
</script>
''')

html_code5

In [13]:
#@title Custom Image
uploaded = files.upload()
fname = list(uploaded.keys())[0]

html_code6 = HTML('''
<!-- 載入tfjs 套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@1.2"></script>
<!-- 載入BodyPix 人體分割模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/body-pix@2.0"></script>

<!-- 顯示原圖的canvas -->
<canvas id='canvas1'></canvas>
<!-- 呈現分析結果的canvas -->
<canvas id='canvas2'></canvas>
<canvas id='canvas3'></canvas>

<script>
// canvas 物件宣告
const canvas1 = document.getElementById('canvas1');
const canvas2 = document.getElementById('canvas2');
const canvas3 = document.getElementById('canvas3');
// canvas 繪製物件宣告
var ctx = canvas1.getContext('2d')

// js 影像物件宣告
var myImage = new Image();
// 指定本地端範例影像路徑
myImage.src = "%s";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // canvas 繪製原圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const net = await bodyPix.load();

  // 模型推論分析
  const partSegmentation = await net.segmentMultiPersonParts(canvas1);
  console.log(partSegmentation);
  
  // 繪製模型分析結果
  const coloredPartImage = bodyPix.toColoredPartMask(partSegmentation);
  const opacity = 0.7; // 底圖影像的不透明度
  const flipHorizontal = false; //關閉水平翻轉前處理
  const maskBlurAmount = 0; // 人體與背景區塊邊界模糊程度
  const pixelCellWidth = 10.0; // 部位pixel-wise寬度

  // 繪製mask結果
  bodyPix.drawMask(
    canvas2, canvas1, coloredPartImage, opacity, maskBlurAmount, flipHorizontal);

  // 繪製結果
  bodyPix.drawPixelatedMask(
    canvas3, canvas1, coloredPartImage, opacity, maskBlurAmount, flipHorizontal, pixelCellWidth);
}
</script>
''' % (fname))

html_code6

Saving 1.jpeg to 1 (2).jpeg


In [14]:
#@title WebCam Image
html_code7 = HTML('''
<!-- 載入tfjs 套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@1.2"></script>
<!-- 載入BodyPix 人體分割模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/body-pix@2.0"></script>

<!-- 顯示原圖的canvas -->
<canvas id='canvas1' height="0" width="0"></canvas>
<!-- 呈現分析結果的canvas -->
<canvas id='canvas2' height="0" width="0"></canvas>
<canvas id='canvas3' height="0" width="0"></canvas>

<script>
// canvas 物件宣告
const canvas1 = document.getElementById('canvas1');
const canvas2 = document.getElementById('canvas2');
const canvas3 = document.getElementById('canvas3');
// canvas 繪製物件宣告
var ctx = canvas1.getContext('2d')

// webcam 攝影
async function takePhoto() {
  // 讀取模型
  const net = await bodyPix.load();

  const div = document.createElement('div'); // 空行
  const capture = document.createElement('button'); // 攝影按鈕
  capture.textContent = 'Capture'; // 攝影按鈕文字
  div.appendChild(capture);

  const video = document.createElement('video'); // 宣告影片物件
  video.style.display = 'block';
  const stream = await navigator.mediaDevices.getUserMedia({video: true}); // 宣告串流物件

  document.body.appendChild(div);
  div.appendChild(video);
  video.srcObject = stream; // 使影片物件顯示串流內容
  await video.play();

  // 調整Video Element 大小
  google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

  // 等待點擊攝影按鈕
  await new Promise((resolve) => capture.onclick = resolve);

  // canvas 繪製底圖
  canvas1.width = video.videoWidth;
  canvas1.height = video.videoHeight;
  ctx.drawImage(video, 0, 0);

  // 停止 webcamera
  stream.getVideoTracks()[0].stop();
  div.remove();

  // 模型推論分析
  const partSegmentation = await net.segmentMultiPersonParts(canvas1);
  console.log(partSegmentation);

  // 繪製模型分析結果
  const coloredPartImage = bodyPix.toColoredPartMask(partSegmentation);
  const opacity = 0.7; // 底圖影像的不透明度
  const flipHorizontal = false; //關閉水平翻轉前處理
  const maskBlurAmount = 0; // 人體與背景區塊邊界模糊程度
  const pixelCellWidth = 10.0; // 部位pixel-wise寬度

  // 繪製mask結果
  bodyPix.drawMask(
    canvas2, canvas1, coloredPartImage, opacity, maskBlurAmount, flipHorizontal);

  // 繪製結果
  bodyPix.drawPixelatedMask(
    canvas3, canvas1, coloredPartImage, opacity, maskBlurAmount, flipHorizontal, pixelCellWidth);
}

takePhoto()
</script>
''')

html_code7